In [1]:
import pandas as pd

In [43]:
df = pd.read_csv('./네이버부동산_5101.csv')

In [45]:
df1 = pd.read_csv('./네이버부동산_5701.csv')

In [1]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()

### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)
            

In [2]:
### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 수집 안 된 법정동
info_df = pd.DataFrame() # 최종 데이터프레임

# json to dataframe (법정동 단위 수집)
for i in range(0,3):
    
    # url 설정
    url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(lawd_cd_unique[i])
      
    try:
        # url 호출
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 데이터프레임화
        df = pd.DataFrame(items)
        info_df = info_df.append(df)
        time.sleep(random.randrange(1,7))

    except:
        fails.append([i])
    
#     # 매물정보(cpLinkVO) 데이터 타입 변경 (dict to list)
#     info_df = info_df.drop_duplicates('atclNo').reset_index()
    
#     for i in info_df.index:
#         for key, value in info_df['cpLinkVO'].iteritems():
#             tmp = [key,value]
#             info_df['cpLinkVO'][i] = tmp
    
#     ### 3. DB INSERT
#     df = info_df
#     table = 'm1.asking_price'

#     # execute_mogrify
#     def execute_mogrify(conn, df, table, val):
#         # Create a list of tuples from the dataframe values
#         tuples = [tuple(x) for x in df.to_numpy()]
#         # Comma-separated dataframe columns
#         cols = ','.join(list(df.columns))
#         # SQL query to execute
#         cursor = conn.cursor()
#         values = [cursor.mogrify(val, tup).decode('utf8') for tup in
#                   tuples]
#         query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

#         try:
#             cursor.execute(query, tuples)
#             conn.commit()
#         except (Exception, psycopg2.DatabaseError) as error:
#             print("Error: %s" % error)
#             conn.rollback()
#             cursor.close()
#             return 1
#         cursor.close()

#     # values 설정
#     val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

#     # insert
#     for j in range(0, len(df), 10000):

#         # DB Connect
#         conn = psycopg2.connect(
#             host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
#             port=5439,
#             dbname='dev',
#             user='awsuser',
#             password='cremaoAdmin1234qwer!!'
#         )

#         # 10000개 단위로 나누기
#         tmp = df[j:j + 10000]

#         # insert
#         execute_mogrify(conn, tmp, table, val)
#         print(datetime.datetime.now(), ' : ', j)
#         tmp = pd.DataFrame()

#     # row count
#     print(len(df))

In [6]:
info_df.columns

Index(['atclNo', 'cortarNo', 'atclNm', 'atclStatCd', 'rletTpCd', 'uprRletTpCd',
       'rletTpNm', 'tradTpCd', 'tradTpNm', 'vrfcTpCd', 'flrInfo', 'prc',
       'rentPrc', 'hanPrc', 'spc1', 'spc2', 'direction', 'atclCfmYmd', 'lat',
       'lng', 'atclFetrDesc', 'tagList', 'bildNm', 'minute', 'sameAddrCnt',
       'sameAddrDirectCnt', 'cpid', 'cpNm', 'cpCnt', 'rltrNm', 'directTradYn',
       'minMviFee', 'maxMviFee', 'etRoomCnt', 'tradePriceHan',
       'tradeRentPrice', 'tradeCheckedByOwner', 'cpLinkVO', 'dtlAddrYn',
       'dtlAddr', 'sameAddrHash', 'sameAddrMaxPrc', 'sameAddrMaxPrc2',
       'sameAddrMinPrc', 'sameAddrMinPrc2', 'repImgUrl', 'repImgTpCd',
       'repImgThumb'],
      dtype='object')

In [20]:
info_df_ = info_df

In [23]:
info_df_.loc[0]

,atclNo,cortarNo,atclNm,atclStatCd,rletTpCd,uprRletTpCd,rletTpNm,tradTpCd,tradTpNm,vrfcTpCd,...,dtlAddr,point,sameAddrHash,sameAddrMaxPrc,sameAddrMaxPrc2,sameAddrMinPrc,sameAddrMinPrc2,repImgUrl,repImgTpCd,repImgThumb
0,2233873259,2717010900,일반상가,R0,D02,D02,상가,B2,월세,OWNER,...,,920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2233873259,2717010900,일반상가,R0,D02,D02,상가,B2,월세,OWNER,...,,920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2233873259,2717010900,일반상가,R0,D02,D02,상가,B2,월세,OWNER,...,,920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2233702436,2720010300,일반상가,R0,D02,D02,상가,B2,월세,SITE,...,,1100,NaN,NaN,NaN,NaN,NaN,/20220930_35/land_naver_1664530206808aBvN3_JPE...,SITE,f130_98
0,2233702436,2720010300,일반상가,R0,D02,D02,상가,B2,월세,SITE,...,,1100,NaN,NaN,NaN,NaN,NaN,/20220930_35/land_naver_1664530206808aBvN3_JPE...,SITE,f130_98
0,2233702436,2720010300,일반상가,R0,D02,D02,상가,B2,월세,SITE,...,,1100,NaN,NaN,NaN,NaN,NaN,/20220930_35/land_naver_1664530206808aBvN3_JPE...,SITE,f130_98
0,2234300460,2717010700,일반상가,R0,D02,D02,상가,B2,월세,DOC,...,,810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2234300460,2717010700,일반상가,R0,D02,D02,상가,B2,월세,DOC,...,,810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2234300460,2717010700,일반상가,R0,D02,D02,상가,B2,월세,DOC,...,,810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2233375335,2717010800,단지내상가,R0,D02,D02,상가,A1,매매,DOC,...,,650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i in info_df_.index:
    for key, value in info_df_['cpLinkVO'].iteritems():
        tmp = [key,value]
        info_df_['cpLinkVO'][i] = tmp

### 자동화

In [ ]:
### 0. 임포트
import json
import time
import random
import requests
import datetime
import psycopg2
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent
ua = UserAgent()


### 1. 법정동 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:10])
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)


### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 수집 안 된 법정동
info_df = pd.DataFrame() # 최종 데이터프레임

# json to dataframe (법정동 단위 수집)
for i in range(0,20551):
    cortar_url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(lawd_cd_unique[i])
    count = 0
    
    # 1~3 페이지 수집
    for num in range(1,4):
        try:
            count += 1
            start = time.time()
            
            # url 호출
            url = cortar_url + str(num)
            headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
            time.sleep(random.randrange(3,7))
            response = requests.get(url, headers = headers)
            
            # body 태그 선택
            items = response.json()['body']
            
            # 데이터프레임화
            df = pd.DataFrame(items)
            info_df = info_df.append(df)
        except:
            fails.append([i,num])
    
    # 수집된 법정동먼저 DB INSERT
    if i % 100 == 0:
        
        ### 3. DB INSERT
        df = info_df
        table = 'm1.asking_price' # 테이블 지정

        # execute_mogrify
        def execute_mogrify(conn, df, table, val):
            # Create a list of tuples from the dataframe values
            tuples = [tuple(x) for x in df.to_numpy()]
            # Comma-separated dataframe columns
            cols = ','.join(list(df.columns))
            # SQL query to execute
            cursor = conn.cursor()
            values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                      tuples]
            query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

            try:
                cursor.execute(query, tuples)
                conn.commit()
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
            cursor.close()

        # values 설정
        val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

        # insert
        for j in range(0, len(df), 10000):

            # DB Connect
            conn = psycopg2.connect(
                host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                port=5439,
                dbname='dev',
                user='awsuser',
                password='cremaoAdmin1234qwer!!'
            )

            # 10000개 단위로 나누기
            tmp = df[j:j + 10000]

            # insert
            execute_mogrify(conn, tmp, table, val)
            print(datetime.datetime.now(), ' : ', j)
            tmp = pd.DataFrame()

        # row count
        print(len(df))